In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Apr_PM_ext");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/East_Massachusetts_trips_Apr_PM_ext.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands;

In [7]:
odPairLabel_;

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec;

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute;

In [12]:
deg = 8
c = 0.5
lam = 10000.0

coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM_ext.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

# fcoeffs = [1, 0, 0, 0, 0.15, 0, 0, 0, 0]

9-element Array{Any,1}:
  1.0       
 -0.00184071
  0.0439163 
 -0.125675  
  0.338971  
 -0.347539  
  0.311807  
 -0.0808905 
  0.0171817 

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 462
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
    free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
    free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:      462
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:      536
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
 512.23       
 457.408      
 796.742      
 854.738      
 212.639      
 280.528      
 614.282      
 470.599      
 531.818      
 662.889      
 301.177      
 404.758      
 272.188      
   ⋮          
 385.238      
   4.1732     
 272.188      
 388.948      
  -4.27046e-33
   3.00927e-36
  -5.47382e-48
   6.84228e-49
 352.83       
 618.87       
 458.628      
 214.308      

In [15]:
getObjectiveValue(m)

3850.562384784591

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 618.870006
  (3,6)   => 531.818372
  (21,13) => 221.21096799999998
  (22,16) => 3.0092655381050547e-36
  (19,12) => 4.728925487330935e-38
  (22,14) => -5.473822126268817e-48
  (15,17) => 352.82973599999997
  (10,11) => 525.585615
  (13,12) => 0.0
  (3,1)   => 854.7377139999999
  (16,13) => 4.173198
  (4,5)   => 347.9693
  (13,16) => 385.23751100000004
  (17,16) => 214.308282
  (20,13) => 201.64533
  (12,8)  => 399.674611
  (2,4)   => 614.2819549999999
  (16,17) => 458.62785199999996
  (1,2)   => 512.230294
  (14,21) => 221.21096799999998
  (2,3)   => 212.638863
  (13,19) => 1.0947644252537633e-47
  (13,14) => 0.0
  (16,14) => 388.948242
  (8,12)  => 482.93854099999993
  ⋮       => ⋮

In [17]:
# getValue(linkFlow)

In [18]:
# getObjectiveValue(m)

In [19]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

No dicts found; please check load_dicts...


In [20]:
flow_user = GLS_Apr_weekday_PM_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [21]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [22]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

for i in week_day_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

4.583763043863958
5.683441547290471
4.861467665793238
5.542897448251787
2.5818131745008635
4.3753834025932195
5.085400354017966
4.972688703262723
5.58115867069303
4.921017879358232
3.7388748981573787
4.74859189269098
4.075150161626218
3.527028462705888
2.5706010159888666
7.360926667596167
4.972994506597086
4.984183044676528
4.913858904253428
3.558146240732754
4.570289635692393
